In [ ]:
import random
import itertools
import pandas as pd

In [2]:
def generate_random_play(deck, n_players, min_cards = 1, max_cards = 5):

    n_cards = random.randint(n_players*min_cards, n_players*max_cards)
    play = random.sample(deck, n_cards)

    return n_cards, n_players, play


def find_combinations(play):

    combs = {
        "one pair":         0,
        "two pairs":        0,
        "three of a kind":  0,
        "straight":         0,
        "flush":            0,
        "full house":       0,
        "four of a kind":   0,
        "straight flush":   0,
    }

    combs["one pair"]        = find_one_pair(play)
    combs["two pairs"]       = find_two_pairs(play)
    combs["three of a kind"] = find_three_of_a_kind(play)
    combs["straight"]        = find_straight(play)
    combs["flush"]           = find_flush(play)
    combs["full house"]      = find_full_house(play)
    combs["four of a kind"]  = find_four_of_a_kind(play)
    combs["straight flush"]  = find_straight_flush(play)

    return combs


def find_one_pair(play):

    play.sort(key = lambda x: x[0])
    
    for _, group in itertools.groupby(play, lambda x: x[0]): 
        if len(list(group)) > 1: return 1

    return 0       


def find_two_pairs(play):

    if len(play) < 4: return 0

    play.sort(key = lambda x: x[0])

    pairs = 0

    for _, group in itertools.groupby(play, lambda x: x[0]): 
        if len(list(group)) > 1: pairs += 1

    return 1 if pairs > 1 else 0


def find_three_of_a_kind(play):

    if len(play) < 3: return 0

    play.sort(key = lambda x: x[0])

    for i in range(len(play)):
        if play[i-2][0] == play[i-1][0] == play[i][0]: return 1

    return 0


def find_straight(play):

    play = list(set(c[0] for c in play))
    play.sort()
    
    if len(play) < 5: return 0

    for i in range(len(play)):
        if play[i-4] == play[i] - 4: return 1

    return 0


def find_flush(play):

    if len(play) < 5: return 0

    play.sort(key = lambda x: x[0])

    groups = []
    
    for _, group in itertools.groupby(play, lambda x: x[0]): 
        groups.append(len(list(group)))

    groups.sort(reverse=True)

    if groups[0] > 2 and groups[1] > 1: return 1

    return 0


def find_full_house(play):

    if len(play) < 5: return 0

    play.sort(key = lambda x: x[1])

    for _, group in itertools.groupby(play, lambda x: x[1]): 
        if len(list(group)) > 4: return 1

    return 0


def find_four_of_a_kind(play):

    if len(play) < 4: return 0

    play.sort(key = lambda x: x[0])
    
    for _, group in itertools.groupby(play, lambda x: x[0]): 
        if len(list(group)) > 3: return 1

    return 0


def find_straight_flush(play):

    if len(play) < 5: return 0

    play.sort(key = lambda x: x[1])

    for _, group in itertools.groupby(play, lambda x: x[1]): 
        subplay = list(group)
        if find_straight(subplay) == 1: return 1

    return 0

In [7]:
values = range(5, 15)
suits = "HDCS"

deck = [(v, s) for v in values for s in suits]

hand_players = [2, 3, 4, 5, 6]

df = pd.DataFrame(columns=[
    "active_players",
    "active_cards",
    "one pair",
    "two pairs",
    "three of a kind",
    "straight",
    "flush",
    "full house",
    "four of a kind",
    "straight flush"])

In [10]:
sample_size = 10000

for hp in hand_players:

    for _ in range(sample_size):
        n_cards, n_players, play = generate_random_play(deck, hp)
        results = find_combinations(play)

        results.update({
            "active_players": n_players,
            "active_cards": n_cards,
        })

        df.loc[len(df)] = results

In [12]:
df.groupby("active_players").mean()

,active_cards,one pair,two pairs,three of a kind,straight,flush,full house,four of a kind,straight flush
active_players,,,,,,,,,
2,6.027489,0.665934,0.345262,0.127949,0.107657,0.088764,0.045082,0.005298,0.002199
3,9.018693,0.839164,0.623950,0.358657,0.328469,0.331767,0.265094,0.037885,0.018693
4,12.017193,0.917833,0.768393,0.561076,0.525890,0.542383,0.493603,0.124150,0.081168
5,14.927229,0.959916,0.850860,0.684626,0.657037,0.667133,0.627449,0.266493,0.196721
6,17.948421,0.984906,0.920832,0.776389,0.757197,0.764294,0.736006,0.414634,0.347861
